In [ ]:
# idk where to put this lol
# also im just messing around here

In [ ]:
import traceback
import inspect

In [ ]:
#THROWING SHIT AT THE WALL
x = 5
def ahhh(y):
    x = y + 4
    #print(dir())
    #print(inspect.trace())
    #print(inspect.stack())
    print(inspect.currentframe().f_back)
    print(inspect.currentframe())
    return 8
#print(globals())
#print(dir())
#print(locals())
#ahhh(8)
#traceback.extract_tb()
#frame_infos = inspect.trace()
#print(inspect.trace())
#ahhh(8)
#print(len(inspect.stack()))
#print(inspect.stack())
#print(inspect.currentframe())
#aaa = inspect.getframeinfo(inspect.currentframe())
#print(type(aaa))
#print(aaa)
#print(inspect.getinnerframes(aaa))
stack = inspect.stack()
outer_frames = inspect.getouterframes(inspect.currentframe())
inner_frames = [frame for frame in stack if frame not in outer_frames]
print(inner_frames)
#print(current_frame)

[FrameInfo(frame=<frame at 0x106e4b380, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/1460007230.py', line 25, code <module>>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/1460007230.py', lineno=25, function='<module>', code_context=['stack = inspect.stack()\n'], index=0, positions=Positions(lineno=25, end_lineno=25, col_offset=8, end_col_offset=23)), FrameInfo(frame=<frame at 0x106e51360, file '/Users/Noemi/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py', line 3530, code run_code>, filename='/Users/Noemi/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py', lineno=3505, function='run_code', code_context=['                    exec(code_obj, self.user_global_ns, self.user_ns)\n'], index=0, positions=Positions(lineno=3505, end_lineno=3505, col_offset=20, end_col_offset=69))]


In [35]:
x = 5
def covid(y):
    x = y + 4
    stack = inspect.stack()
    outer_frames = inspect.getouterframes(inspect.currentframe())
    inner_frames = [frame for frame in stack if frame not in outer_frames]
    print(len(inner_frames))
    print(inner_frames)
    print("wehjfbhjewfbjhwefhjb")
    print(inspect.currentframe())
    def ah2(z):
        return z + y
    return ah2
#print(globals())
#print(dir())
#print(locals())
g = covid(8)

stack = inspect.stack()
outer_frames = inspect.getouterframes(inspect.currentframe())
inner_frames = [frame for frame in stack if frame not in outer_frames]
print(len(inner_frames))
print(inner_frames)
print(outer_frames)

1
[FrameInfo(frame=<frame at 0x106f50200, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/82531716.py', line 8, code covid>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/82531716.py', lineno=4, function='covid', code_context=['    stack = inspect.stack()\n'], index=0, positions=Positions(lineno=4, end_lineno=4, col_offset=12, end_col_offset=27))]
wehjfbhjewfbjhwefhjb
<frame at 0x106f50200, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/82531716.py', line 10, code covid>
2
[FrameInfo(frame=<frame at 0x105440f40, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/82531716.py', line 19, code <module>>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/82531716.py', lineno=19, function='<module>', code_context=['stack = inspect.stack()\n'], index=0, positions=Positions(lineno=19, end_lineno=19, col_offset=8, end_col_offset=23)), FrameInfo(frame=<frame at 0x106e518b0, file '/